In [1]:
import pandas as pd
import tensorflow as tf
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

# Carregar os dados dos filmes
df = pd.read_csv("dados.csv")  # Certifique-se de ter um arquivo CSV com os dados dos filmes

# Pré-processamento dos dados
# Aqui vamos usar apenas título e sinopse como features
text_data = df['Titulo'] + " " + df['Sinopse']

# Vetorização dos dados de texto
vectorizer = TfidfVectorizer(stop_words='english')
text_matrix = vectorizer.fit_transform(text_data)

# Modelo de similaridade
def recommend_movies(input_text, vectorizer, text_matrix, df, top_n=5):
    # Vetorizar o texto de entrada
    input_vector = vectorizer.transform([input_text])
    # Calcular a similaridade do cosseno entre o texto de entrada e os textos dos filmes
    similarities = cosine_similarity(input_vector, text_matrix)
    # Obter os índices dos filmes mais semelhantes
    similar_indices = similarities.argsort(axis=1)[:, ::-1][:, :top_n]
    # Recuperar os filmes recomendados
    recommended_movies = df.iloc[similar_indices[0]]
    return recommended_movies

# Teste do programa
user_input = "Naruto"
recommended_movies = recommend_movies(user_input, vectorizer, text_matrix, df)
recommended_movies[['Titulo', 'Sinopse']]

,Titulo,Sinopse
22,Naruto Shippuden: The Movie,the adventures of adolescent ninja naruto uzum...
26,Naruto the Movie: Ninja Clash in the Land of Snow,naruto sasuke and sakura learn they ll be pro...
20,Naruto Shippûden the Movie: Bonds,when strange ninjas ambush the village of kono...
23,Naruto Shippuden: The Movie: The Lost Tower,when naruto is sent to recover a missing nin ...
4707,Naruto Shippuden : Blood Prison,mistakenly accused of an attack on the fourth ...
